<a href="https://colab.research.google.com/github/rohitkhadka1/FxPy/blob/main/Backtesting_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook containing my practice exercises which I did while learning a python framework for backtesting trading strategies

In [2]:
!pip install backtesting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 4.5 MB/s eta 0:00:00


In [3]:
from backtesting.test import EURUSD

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [4]:
EURUSD.head()

,Open,High,Low,Close,Volume
2017-04-19 09:00:00,1.07160,1.07220,1.07083,1.07219,1413
2017-04-19 10:00:00,1.07214,1.07296,1.07214,1.07260,1241
2017-04-19 11:00:00,1.07256,1.07299,1.07170,1.07192,1025
2017-04-19 12:00:00,1.07195,1.07280,1.07195,1.07202,1460
2017-04-19 13:00:00,1.07200,1.07230,1.07045,1.07050,1554


In [5]:
EURUSD.tail()

,Open,High,Low,Close,Volume
2018-02-07 11:00:00,1.23390,1.23548,1.23386,1.23501,2203
2018-02-07 12:00:00,1.23501,1.23508,1.23342,1.23422,2325
2018-02-07 13:00:00,1.23422,1.23459,1.23338,1.23372,2824
2018-02-07 14:00:00,1.23374,1.23452,1.23238,1.23426,4065
2018-02-07 15:00:00,1.23427,1.23444,1.22904,1.22904,6143


For the backtesting purpose we need a trading strategy right. so let us create a simple moving average cross-over strategy

In [6]:
import pandas as pd
def SMA(values, n):
  """
  Return simple moving average of given list of values
  """
  return pd.Series(values).rolling(n).mean()

A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().

Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, vectorized manner whatever indicators and signals the strategy depends on.

Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), simulating the incremental availability of each new full candlestick bar.

Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the next candle's open (or the current candle's close if trade_on_close=True). If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin with more fine-grained (e.g. hourly) data.

In [7]:
from backtesting import Strategy
from backtesting.lib import crossover

class SMACrossOver(Strategy):
  n1 = 10
  n2 = 20

  def init(self):
    # Precompute the two moving averages
    self.sma1 = self.I(SMA, self.data.Close, self.n1)
    self.sma2 = self.I(SMA, self.data.Close, self.n2)

  def next(self):
    # If sma1 crosses over sma2, close any existing
    # short trades, and buy the asset
    if crossover(self.sma1, self.sma2):
      self.position.close()
      self.buy()

    # Else, if sma2 crosses over sma1, close any existing
    # long trades, and sell the asset
    elif crossover(self.sma2, self.sma1):
      self.position.close()
      self.sell()

In [ ]:
%%script echo
   def next(self):
       if (self.sma1[-2] < self.sma2[-2] and self.sma1[-1] > self.sma2[-1]):
           self.position.close()
           self.buy()
       elif (self.sma1[-2] > self.sma2[-2] and self.sma1[-1] < self.sma2[-1]):
             self.position.close()
             self.sell()

Backtesting

In [12]:
from backtesting import Backtest

bt = Backtest(EURUSD, SMACrossOver, cash = 10000, commission = 0.002)
stats = bt.run()
stats

Backtest.run:   0%|          | 0/4980 [00:00<?, ?bar/s]

,0
Start,2017-04-19 09:00:00
End,2018-02-07 15:00:00
Duration,294 days 06:00:00
Exposure Time [%],99.18
Equity Final [$],3526.917959
Equity Peak [$],10006.117523
Commissions [$],6341.564264
Return [%],-64.73082
Buy & Hold Return [%],14.56803
Return (Ann.) [%],-64.876956


In [13]:
bt.plot()

GridPlot(id='p1349', ...)

Optimization

In [14]:
%%time

stats = bt.optimize(n1=range(5,30,5),
                    n2 = range(10,70,5),
                    maximize = 'Equity Final [$]',
                    constraint = lambda param: param.n1 < param.n2)
stats

Backtest.optimize:   0%|          | 0/50 [00:00<?, ?it/s]

Backtest.run:   0%|          | 0/4935 [00:00<?, ?bar/s]

CPU times: user 1.22 s, sys: 75.1 ms, total: 1.3 s
Wall time: 8.77 s


,0
Start,2017-04-19 09:00:00
End,2018-02-07 15:00:00
Duration,294 days 06:00:00
Exposure Time [%],95.9
Equity Final [$],7245.436135
Equity Peak [$],10000.0
Commissions [$],2712.294709
Return [%],-27.545639
Buy & Hold Return [%],13.374844
Return (Ann.) [%],-27.63859


In [15]:
stats._strategy

<Strategy SMACrossOver(n1=20,n2=65)>

In [16]:
bt.plot(plot_volume = False, plot_pl = False)

GridPlot(id='p1618', ...)

In [17]:
stats['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
2017-04-19 09:00:00,10000.000000,0.000000,NaT
2017-04-19 10:00:00,10000.000000,0.000000,NaT
2017-04-19 11:00:00,10000.000000,0.000000,NaT
2017-04-19 12:00:00,10000.000000,0.000000,NaT
2017-04-19 13:00:00,10000.000000,0.000000,NaT
...,...,...,...
2018-02-07 11:00:00,7211.150425,0.278885,NaT
2018-02-07 12:00:00,7215.687395,0.278431,NaT
2018-02-07 13:00:00,7218.558895,0.278144,NaT
2018-02-07 14:00:00,7215.457675,0.278454,NaT


In [18]:
stats['_trades'] # Contains individual trade data

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_SMA(C,20)","Exit_SMA(C,20)","Entry_SMA(C,65)","Exit_SMA(C,65)"
0,-9187,149,180,1.08624,1.09130,None,None,-46.48622,-0.004658,2017-04-27 14:00:00,2017-04-30 21:00:00,3 days 07:00:00,None,1.089989,1.089699,1.090158,1.089379
1,9065,180,254,1.09130,1.08890,None,None,-21.75600,-0.002199,2017-04-30 21:00:00,2017-05-03 23:00:00,3 days 02:00:00,None,1.089699,1.090876,1.089379,1.091178
2,-9029,254,273,1.08890,1.09675,None,None,-70.87765,-0.007209,2017-05-03 23:00:00,2017-05-04 18:00:00,0 days 19:00:00,None,1.090876,1.092254,1.091178,1.091752
3,8864,273,321,1.09675,1.09287,None,None,-34.39232,-0.003538,2017-05-04 18:00:00,2017-05-08 18:00:00,4 days 00:00:00,None,1.092254,1.095438,1.091752,1.095921
4,-8829,321,414,1.09287,1.09214,None,None,6.44517,0.000668,2017-05-08 18:00:00,2017-05-12 15:00:00,3 days 21:00:00,None,1.095438,1.087772,1.095921,1.087515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,5873,4525,4641,1.19574,1.22285,None,None,159.21703,0.022672,2018-01-10 21:00:00,2018-01-17 17:00:00,6 days 20:00:00,None,1.196029,1.224333,1.195881,1.224602
78,-5849,4641,4674,1.22285,1.22442,None,None,-9.18293,-0.001284,2018-01-17 17:00:00,2018-01-19 02:00:00,1 days 09:00:00,None,1.224333,1.223472,1.224602,1.223167
79,5811,4674,4823,1.22442,1.24158,None,None,99.71676,0.014015,2018-01-19 02:00:00,2018-01-29 07:00:00,10 days 05:00:00,None,1.223472,1.242217,1.223167,1.242593
80,-5788,4823,4866,1.24158,1.24146,None,None,0.69456,0.000097,2018-01-29 07:00:00,2018-01-31 02:00:00,1 days 19:00:00,None,1.242217,1.240832,1.242593,1.240298
